# Batch Process multiple freemocap session folders


In [ ]:
#import stuff
from pathlib import Path
from typing import Union

from src.core_processes.batch_process_session_folders import process_session_folder

print('a')
from src.blender_stuff.create_blend_file_from_session_data import create_blend_file_from_session_data
print('b')
from src.core_processes.capture_volume_calibration.anipose_camera_calibration import freemocap_anipose
print('c')
from src.core_processes.capture_volume_calibration.triangulate_3d_data import triangulate_3d_data
print('d')
from src.core_processes.mediapipe_stuff.convert_mediapipe_npy_to_csv import convert_mediapipe_npy_to_csv
print('e')
from src.core_processes.mediapipe_stuff.mediapipe_skeleton_detector import MediaPipeSkeletonDetector
print('f')
from src.core_processes.post_process_skeleton_data.gap_fill_filter_and_origin_align_skeleton_data import    gap_fill_filter_origin_align_3d_data_and_then_calculate_center_of_mass
print('g')

In [ ]:
# getcher paths straight
path_to_folder_of_session_folders = Path(
    r'D:\Dropbox\Northeastern\Courses\Biol2299\2022_09_Fall_Biol2299_Inquiries_NeuralControlOfRealWorldBehavior\freemocap_bos_com_standing_data\FreeMocap_Data')

path_to_camera_calibration_toml = Path(
    r"D:\Dropbox\Northeastern\Courses\Biol2299\2022_09_Fall_Biol2299_Inquiries_NeuralControlOfRealWorldBehavior\freemocap_bos_com_standing_data\FreeMocap_Data\sesh_2022-09-28_15_57_08_calibration\sesh_2022-09-28_15_57_08_calibration_calibration.toml"
)

path_to_blender_executable = Path(r"C:\Program Files\Blender Foundation\Blender 3.2\blender.exe")

In [ ]:
# get anipose calibration object
anipose_calibration_object = freemocap_anipose.CameraGroup.load(str(path_to_camera_calibration_toml))

In [ ]:
#get paths to folder full of session folders
list_of_session_folders = list(path_to_folder_of_session_folders.glob('ses*'))
print(list_of_session_folders)

In [ ]:
session_type = 'pre-alpha'
for session_folder in list_of_session_folders:
    print(f'Processing {session_folder}')
    if session_type == 'pre-alpha':
        synchronized_videos_folder = Path(session_folder) / 'SyncedVideos'
    else:
        synchronized_videos_folder = Path(session_folder) / 'synchronized_videos'

    output_data_folder = Path(session_folder) / 'output_data'
    output_data_folder.mkdir(exist_ok=True, parents=True)
    process_session_folder(synchronized_videos_folder=synchronized_videos_folder,
                           output_data_folder=output_data_folder,)

print("Done!")

In [ ]:
489*553

In [ ]:
6052*553